In [1]:
import os
os.environ['HTTP_PROXY'] = "127.0.0.1:10809"
os.environ['HTTPS_PROXY']="127.0.0.1:10809"
os.environ["OPENAI_API_KEY"] = "YOU-OPENAI-KEY"

In [2]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0.1)

In [3]:
import pdfplumber

def read_pdf(file_path, n):
    inner = []
    text = ""
    with pdfplumber.open(file_path) as pdf:
            # 提取前n页
            for i, page in enumerate(pdf.pages):
                # 如果已经处理了n页，就停止
                if i == n:
                    break
                inner.append(page.extract_text())
    return inner

In [4]:
content = read_pdf(r'./data/KT820说明书V2.0 201909.pdf', 30)

In [5]:
inner = []
inner.append('\n'.join(content[:10]))
inner.append('\n'.join(content[10:20]))
inner.append('\n'.join(content[25:30]))


In [6]:
from langchain.prompts import  ChatPromptTemplate
prompts = """
    ----
    {text}
    ----
    帮我把上面内容总结成2-4个目录，目录不要子目录，目录为简洁摘要，其它内容不用给我
    """

prompts2 = """
    abstract:
    ----
    {abstract}
    ----
    text:
    ----
    {text}
    ----

    跟据上面的摘要，找出下面内容中每个摘要对应的开始位置内容：
    返回格式:
    {respond_struction}

"""

prompts3 = """
    text:
    ----
    {text}
    ----

    Json
    '''
    {Json}
    '''
    根据上面的json的内容,从text中找到对应的段落并且返回符合逻辑的内容回来,text用---分割,json用'''来分割

    返回格式:
    {respond_struction}

"""
promptsTemplate = ChatPromptTemplate.from_template(prompts)
promptsTemplate2 = ChatPromptTemplate.from_template(prompts2)
promptsTemplate3 = ChatPromptTemplate.from_template(prompts3)

## 指定返回值格式
> 返回值格式采用json数据,包含三部分:关键词,开始词,结束词
### langchain的json格式
1. 先定义单个模式,Schema = ResponseSchema(name="",description="")
2. 然后将多个模式组合在一起response_schemas = [abstract_schema, start_schema, end_schema]
3. 通过结构输出解析函数解析成结构对象 = StructuredOutputParser(response_schemas = response_schemas)
4. 最后将结构对象解析成json格式的内容


In [7]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
abstract_schema = ResponseSchema(name="abstract",description="这是摘要部分的内容, 也就是用来记录摘要的")
content_schema = ResponseSchema(name="content",description="返回和上面摘要部分相关的全部内容,有n个摘要就返回n个内容")

response_schemas = [abstract_schema, content_schema]
out_parse = StructuredOutputParser(response_schemas = response_schemas)
format_instructions = out_parse.get_format_instructions()

## 制作两条链来定位
* 第一条链用来摘要总结文本内容,第一条链用3.5的模型
* 第二条链用来根据摘要来定位文本内容,第二条链用3.5模型

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=promptsTemplate)
respones=chain.run(text = inner[2])

In [9]:
# llm2 = ChatOpenAI(model_name='gpt-4-0613', temperature=0.2)
llm2 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0.1)
chain2 = LLMChain(llm=llm2, prompt=promptsTemplate2)
result = chain2.run(text=inner[2],abstract=respones,respond_struction=format_instructions)
#

In [10]:
print(respones)

1. G31指令：用于检测输入口信号并控制程序跳转
2. G32指令：用于切削直螺纹和锥螺纹
3. G33指令：用于刚性攻丝
4. G34指令：用于变螺距螺纹切削
5. G92指令：用于螺纹切削单一循环


In [11]:
print(result)
print(len(result))
print(len(inner[2]))

```json
{
	"abstract": "G31指令：用于检测输入口信号并控制程序跳转",
	"content": "若在 Z 轴走完 W160 后仍未检测到 8 号输入口的低电平信号，系统结束 G31 段，执行\nG0 U60段。\nZ\n160\nA C B\n实际移动\n第\n一 无skip信号时的移动\n06\n篇\n编\nD D'\n程 skip信号输入点\nX\n篇\n图2-4\n举例2：如下图2－5，轨迹A－B－D为无跳转信号的运行轨迹\n执行 G31 W80 K6 F200\nG01 X300 Z100\n程序执行时，以F200的速度进给Z轴，同时检测6号输入口状态，在走到C点位置时，\n系统检测到6号输入口的高电平信号，程序结束G31段执行，立刻跳转到G01 X300 Z100段\n执行。这样，实际运行轨迹为 A－C－D。\n若Z轴到达B点后仍未检测到6号输入口的高电平信号，系统结束G31段，执行G01 X300\nZ100段。\nZ\nA(100,0) C B(100,80)\nD(300,100)\nskip信号输入点\nX\n图2－5"
}
{
	"abstract": "G32指令：用于切削直螺纹和锥螺纹",
	"content": "用G32指令，可以切削导程不变的直螺纹，锥螺纹。\n直螺纹指令格式：G32 Z(W)__F/I__；\nZ（W）：螺纹终点 Z向位置；\nF：公制螺纹，长轴方向的导程（0.001—500.000mm）。\nI：英制螺纹，长轴方向的每英寸牙数（1—25400牙/英寸）\n锥螺纹指令格式：G32 X（U）__Z(W)__F/I__；\nX（U）：螺纹终点 X向位置；\nZ（W）：螺纹终点 Z向位置；\n22\nF：公制螺纹，长轴方向的导程（0.001—500.000mm）。\nI：英制螺纹，长轴方向的每英寸牙数（1—25400牙/英寸）\n在螺纹切削开始及结束部分，一般由于升降速的原因，会出现导程不正确部分，考虑此\n因素影响，指令螺纹长度应当比需要的螺纹长度要长些。\n例1：直螺纹切削\n68 第\n一\n篇\nZ\n编\n2 1 程\n篇\n52\nX\n在Z方向：△1=3mm，△2=1.5mm\n螺纹导程：3mm\n在牙深共2mm（公制输入，直径编程）：\nG00 U-25.0 //定位，进刀深度为2

1. [{},{},{}]
2. 输出文件